In [17]:
import json
from nltk.corpus import indian
from nltk.tag import tnt 

with open("/home/akanksha/HindiWN_1_5/database/data_txt", "r", encoding = "utf-8") as file:
    wordNetFile = file.read()

wordNetFile = wordNetFile.strip().splitlines()
wordDictionary = []
for sent in wordNetFile:
    wordDictionary.append(sent.split(" ")[3].split(":"))
synonym_dictionary = {}
for i in wordDictionary:
    for j in i:
        if j not in synonym_dictionary:
            synonym_dictionary[j] = set()
        for k in i:
                synonym_dictionary[j].add(k)

In [30]:
inp_file="4.txt"
percentage=0.4

In [23]:
from functools import reduce
from string import punctuation
from nltk import word_tokenize
from collections import Counter
from wxconv import WXC
import subprocess 
import os.path
import json
import copy
import nltk
import math
import sys
import os
import re

with open("support_data/stopwords.txt", 'r', encoding = "utf-8") as stop:
    stopWords = stop.read().splitlines()
    
    
with open("/home/akanksha/Text_Summarizer/amarujala/"+inp_file,"r") as g:
    input_text = g.read().strip()
    
with open("support_data/month_day.txt", 'r', encoding = "utf-8") as mon:
    month_day = mon.read().splitlines()
     
    
def remove_stopwords(text):
    text=[i for i in text if i not in stopWords]
    return(text)


def generateBigrams(tokenized_list):
    bigramsDict = dict()
    bigramsWordsList = list()
    bigram = list()
    bigrams = list(nltk.bigrams(list(reduce(lambda x,y: x+y,tokenized_list))))
    cfd = nltk.ConditionalFreqDist(bigrams)
    for inneritems in cfd.items():
        for items in inneritems[1].items():
            if items[1] > 2:
                if items[1] not in bigramsDict:
                    bigramsDict[items[1]] = []
                    bigramsDict[items[1]].append([inneritems[0], items[0]])
                else:
                    bigramsDict[items[1]].append([inneritems[0], items[0]])
                bigramsWordsList.append([inneritems[0], items[0]])
    return(bigramsDict , bigramsWordsList, bigram)


def countBigrams(sentences): 
    sentenceBigrams = list(nltk.bigrams(sentences))
    count = 0
    for items in sentenceBigrams:
        if list(items) in bigramsWordsList:
            count +=  1
    return count/len(sent)


def partsOfSpeechTagger(localWordList):
    train_data = indian.tagged_sents('hindi.pos') 
    tnt_pos_tagger = tnt.TnT()
    tnt_pos_tagger.train(train_data)
    return(tnt_pos_tagger.tag(localWordList))


def contentWords(pos_info, flag):
    content = []
    if flag:
        for words in pos_info:
            if not (words[1] == 'VAUX' or words[1] == 'SYM' or words[1] =='VFM' or words[1] == 'CC' or words[1] == 'PRP' or
                words[1] ==  'PUNC' or words[1] ==  'QF' or words[1] ==  'RB' or words[1] ==  'QW' or words[
                1] ==  'RP' or words[1] ==  'PREP'):
                content.append(words[0])
    else:
        for words in pos_info:
            if (words[1] ==  'NN' or words[1] ==  'NNP' or words[1] ==  'Unk'):
                content.append(words[0])
    return(content)


def morph_get(word):
    f1=open("hi","w")
    f1.write(word)
    f1.close()
    cmd="echo $HOME"
    bin_path=subprocess.check_output(cmd, shell=True)
    cmd1="sh "+bin_path.decode("utf-8").strip()+"/bin/hin_morph.sh < hi"
    result=subprocess.check_output(cmd1, shell=True)
    return(result.decode("utf-8"))

def word_(word):
    f1 = open('canonical',"w")
    f1.write(word)
    f1.close()
    cmd="echo $HOME_anu_test"
    path = "/Anu_data/canonical_form_dictionary"
    bin_path=subprocess.check_output(cmd, shell=True)
    cmd1=bin_path.decode("utf-8").strip()+path+"/canonical_form.out < canonical"
    result=subprocess.check_output(cmd1, shell=True)
    return(result.decode("utf8"))


con = WXC(order='utf2wx') 
con1 = WXC(order='wx2utf')

def get_root(word):
    morph_list =  list(set(map(lambda x: x.replace("$_","").split("<")[0].strip("$").strip("*"), morph_get(word_(con.convert(word))).split("/")[1:])))
    final_morph=list(map(lambda x:con1.convert(x), morph_list))
    return(final_morph)
    

def generateCueWordList(titleList):
    cueWordList = []
    for words in titleList:
        if words in synonym_dictionary:
            temp_word = synonym_dictionary[words]
        else:
            temp_word = []
            temp_word1 = get_root(words)
            for tem in temp_word1:
                if tem in synonym_dictionary:
                    temp_word.extend(synonym_dictionary[tem])
        cueWordList.extend(temp_word)
    return(cueWordList)


def titleFeatures(sent, titleList):
    count_title = 0
    for each_word in sent:
        if each_word in titleList:
            count_title+=1
    return(count_title/len(sent))

def cueFeatures(sent, cueWordList):
    count_cue = 0
    for each_word in sent:
        if each_word in cueWordList:
            count_cue +=1
    return(count_cue/len(sent))

def mon_dayFeatures(sent):
    count=0
    for each_word in sent:
        if each_word in month_day:
            count+=1
    return(count/len(sent))

def computeTf(wordDict, sent):
    tfDict={}
    for word in sent:
        wordDict[word]+=1
    length = len(sent)
    for word, count in wordDict.items():
        tfDict[word] = count/float(length)
    return(tfDict, wordDict)

def computeIDF(docList):
    idfDict = {}
    N = len(docList)    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
    return idfDict

def computeTFIDF(tf, idfs):
    tfidf = {}
    for word, val in tf.items():
        tfidf[word] = val*idfs[word]
    return tfidf

def properNounFeature(pos_w):
    count_prop = 0
    count_unk = 0
    for items in pos_w:
        if (items[1] ==  "NNP"):
            count_prop+=1
        if (items[1] ==  "Unk"):
            count_unk+=1
    return(count_prop/len(pos_w),count_unk/len(pos_w))

def dateFeature(sent):
    pattern=re.compile(r'((?:(?:(?:(?:(?:[012]\d{1})|[1-9])[-\/]){2}(?:[12]\d{3}|\d{2}))|(?:[01]?\d{1}[-\/])(?:30|31)[-\/](?:[12]\d{3}|\d{2})|(?:(?:[012]\d{1})|[1-9])[-\/][12]\d{3}|(?:(?:\d{1,2}\s)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:\.|,|\s|-)?(?:\d{1,2})?(?:st|th|rd|nd)?(?:,|\s|-)?\s?[[12]\d{3}|\d{2}])|(?:(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:\.|,|\s|-)?\s?(?:\d{1,2})(?:st|th|rd|nd)?(?:,|\s|-)?\s?[[12]\d{3}|\d{2}]))|[12]{1}\d{3})')
    pat=re.findall(pattern, sent)
    return(len(pat)/len(sent))

def sent_position():
    l=len(tokenized_list)
    position=[0]*l
    if l%2==0:
        mid=int(l/2)
    else:
        mid=int((l+1)/2 )  
    for i in range(0,mid):
        position[i]=l-i
        position[l-i-1]=l-i
    return(position)

def englishLoanwordsFeature(sent):
    count=0
    for i in sent:
        if i in english_loanwords:
            count+=1
    return(count/len(sent))

def hasNumbers(inputString):
     return sum([1 if char.isdigit() else 0 for char in inputString])/len(inputString)

orig_title = input_text.split("\n")[0].strip()
title= remove_stopwords(word_tokenize(orig_title.translate(str.maketrans('', '', punctuation.replace("-","").replace("'","").replace('"',"")))))
titleList = contentWords(partsOfSpeechTagger(title),0)
cueWordList = generateCueWordList(titleList)
rest_text_ = input_text.strip(orig_title)
rest_text= rest_text_.translate(str.maketrans('', '', punctuation)).replace(u"\xa0","")
segmented_text =  rest_text.strip().split("।")
try:
    segmented_text.remove("")
except:
    print("nothing")
    
originalFile = copy.deepcopy(rest_text)
originalSentenceList = [sentenceList.strip() for sentenceList in rest_text_.split("।")]
tokenized_list = [remove_stopwords(word_tokenize(i)) for i in segmented_text]
index_sent =  {" ".join(sen):loc for loc, sen in enumerate(tokenized_list)}
bigramsDict , bigramsWordsList, bigram = generateBigrams(tokenized_list)
bigramCountList = [countBigrams(i) for i in tokenized_list]
title_feature = [titleFeatures(sent, titleList) for sent in tokenized_list]
cue_feature = [cueFeatures(sent,cueWordList) for sent in tokenized_list]
month_day_feature = [mon_dayFeatures(sent) for sent in tokenized_list]
vocab=set(reduce(lambda x,y:x+y,tokenized_list))
wordDict = dict.fromkeys(vocab, 0) 
tfword=[computeTf(wordDict.copy(),sent) for sent in tokenized_list]
docList=[i[1] for i in tfword]
tf=[i[0] for i in tfword]
idfs=computeIDF(docList)
tfidf=[computeTFIDF(tf_i, idfs) for tf_i in tf]
tfidf_average = [sum(j.values())/len([e for e in j.values() if e]) for j in tfidf]
prt_speech = [partsOfSpeechTagger(sent) for sent in tokenized_list]
noun_feature = [properNounFeature(i) for i in prt_speech]
prp_feature= [i[0] for i in noun_feature]
unk_feature= [i[1] for i in noun_feature]
date_feature = [dateFeature(i) for i in segmented_text]
position_feature = sent_position()
eng_loan = [englishLoanwordsFeature(sent) for sent in tokenized_list]
digit = [hasNumbers(sent) for sent in tokenized_list]

In [31]:
import numpy as np
sum_features = [0]*len(tokenized_list)
for loc,sent in enumerate(tokenized_list):
    sum_features[loc] = 10*title_feature[loc]+9*cue_feature[loc]+month_day_feature[loc]+bigramCountList[loc]+tfidf_average[loc]+6*prp_feature[loc]+unk_feature[loc]+8*date_feature[loc]+position_feature[loc]+eng_loan[loc]+digit[loc]
    

sum_features = np.array(sum_features)
ids = (-sum_features).argsort()[:int(np.ceil(len(tokenized_list)*percentage))]
ids=sorted(ids)
summary=[]
for i in ids:
    summary.append(originalSentenceList[i].replace(u"\xa0","")+"।")
    
final_summary = orig_title+"\n"+" ".join(summary)



In [32]:
print(input_text)

एक घंटा 45 मिनट तक खून जैसा लाल दिखेगा चांद, दुनिया देखेगी सदी का सबसे लंबा ग्रहण

आगामी शुक्रवार वह दिन होगा जो सदियों तक याद रखा जाएगा। इस दिन दुनिया सदी का सबसे लंबा चंद्र ग्रहण देखेगी जो एक घंटा 45 मिनट तक रहेगा। यह दिन खगोलशास्त्रियों के लिए भी खास होने जा रहा है। 27 जुलाई की मध्यरात्रि में लगने जा रहे इस खगोलीय घटना में करीब चार घंटे तक चन्द्रमा इस ग्रहण के प्रभाव में रहेगा। इस दिन मंगल भी पृथ्वी के काफी करीब आने वाला है। चंद्र ग्रहण को लेकर पूरे देश के खगोलशास्त्री काफी उत्साहित हैं। 

यह एक ऐसी घटना है जब दुनियाभर के स्टार गैजर को रक्त जैसे लाल चंद्रमा को देखने का मौका मिलेगा। यह स्थिति तब उत्पन्न होती है जब चंद्रमा पूरी तरह से ग्रहण में होता है और सूरज की रोशनी के कारण लाल दिखाई देने लगता है।

अस्ट्रेलियन नेशनल यूनिवर्सिटी के रिसर्च स्कूल ऑफ एस्ट्रोनोमी और एस्ट्रेफिजिक्स के एस्ट्रोनोमर ब्रैड टकर  ने बताया कि चंद्रमा हमेशा सूर्य और पृथ्वी के साथ पूर्ण संरेखण में नहीं होता है इसलिए हमें हर चंद्र चक्र में हमें चंद्र ग्रहण देखने को नहीं मिलता है। उन्होंने आगे कहा कि हम देखते हैं क

In [33]:
print(final_summary)

एक घंटा 45 मिनट तक खून जैसा लाल दिखेगा चांद, दुनिया देखेगी सदी का सबसे लंबा ग्रहण
आगामी शुक्रवार वह दिन होगा जो सदियों तक याद रखा जाएगा। इस दिन दुनिया सदी का सबसे लंबा चंद्र ग्रहण देखेगी जो एक घंटा 45 मिनट तक रहेगा। 27 जुलाई की मध्यरात्रि में लगने जा रहे इस खगोलीय घटना में करीब चार घंटे तक चन्द्रमा इस ग्रहण के प्रभाव में रहेगा। चंद्र ग्रहण को लेकर पूरे देश के खगोलशास्त्री काफी उत्साहित हैं। पृथ्वी के वातावरण की वजह से रोशनी मुड़कर चांद पर पड़ती है और इस वजह से यह लाल नजर आता है। जब पूर्ण चंद्रग्रहण होता है तभी ब्लड मून होता है। इतना लंबा ग्रहण करीब 150 साल बाद दिखाई देने जा रहा है। यहआरंभ रात 11 बजकर 54 मिनट पर होगा इसका मध्यकाल रात 1 बजकर 54 मिनट पर होगा और 28 जुलाई को सुबह 3 बजकर 49 मिनट पर ग्रहण कासमाप्तहोगा। इस तरह ग्रहण की कुल अवधि 3 घंटे 55 मिनट की होगी।


In [363]:
int(np.ceil(len(tokenized_list)*0.4))

4

In [ ]:
title_feature

In [268]:
cue_feature

[0.02702702702702703, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [269]:
month_day_feature

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [270]:
bigramCountList

[0.004329004329004329,
 0.012987012987012988,
 0.0,
 0.004329004329004329,
 0.004329004329004329,
 0.004329004329004329,
 0.004329004329004329,
 0.008658008658008658]

In [271]:
tfidf_average

[0.012133943116785863,
 0.017391387694478136,
 0.04626387040077326,
 0.020283215710739017,
 0.03973620601272673,
 0.04868028327937198,
 0.0513253720776522,
 0.0190496180175016]

In [272]:
prp_feature

[0.013513513513513514, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [273]:
unk_feature

[0.6351351351351351,
 0.5813953488372093,
 0.5882352941176471,
 0.5641025641025641,
 0.55,
 0.8666666666666667,
 0.7272727272727273,
 0.6818181818181818]

In [274]:
date_feature

[0.0, 0.0031746031746031746, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [7]:
nltk.download('indian')

[nltk_data] Downloading package indian to /home/akanksha/nltk_data...
[nltk_data]   Package indian is already up-to-date!


True

In [192]:
import os.path
from wxconv import WXC
import sys
import os
import subprocess
def morph_get(word):
    f1=open("hi","w")
    f1.write(word)
    f1.close()
    cmd="echo $HOME"
    bin_path=subprocess.check_output(cmd, shell=True)
    cmd1="sh "+bin_path.decode("utf-8").strip()+"/bin/hin_morph.sh < hi"
    result=subprocess.check_output(cmd1, shell=True)
    return(result.decode("utf-8"))

def word_(word):
    f1 = open('canonical',"w")
    f1.write(word)
    f1.close()
    cmd="echo $HOME_anu_test"
    path = "/Anu_data/canonical_form_dictionary"
    bin_path=subprocess.check_output(cmd, shell=True)
    cmd1=bin_path.decode("utf-8").strip()+path+"/canonical_form.out < canonical"
    result=subprocess.check_output(cmd1, shell=True)
    return(result.decode("utf8"))



con = WXC(order='utf2wx') 
con1 = WXC(order='wx2utf')

def get_root(word)
    morph_list =  list(set(map(lambda x: x.replace("$_","").split("<")[0].strip("$").strip("*"), morph_get(word_(con.convert(word))).split("/")[1:])))
    final_morph=list(map(lambda x:con1.convert(x), morph_list))
    return(word)

AttributeError: 'list' object has no attribute 'replace'

In [221]:
calculateIdf(tokenized_list)

Counter({'लिखा': 1.3862943611198906,
         'पाया': 2.0794415416798357,
         'नवोदय': 2.0794415416798357,
         'पढ़': 2.0794415416798357,
         'भी': 0.9808292530117262,
         'मेरे': 1.3862943611198906,
         'मरा': 2.0794415416798357,
         'वर्ष': 1.3862943611198906,
         'हां': 2.0794415416798357,
         'गयी': 2.0794415416798357,
         '7': 2.0794415416798357,
         'विद्यालय': 2.0794415416798357,
         'लेकर': 1.3862943611198906,
         'कक्षा': 2.0794415416798357,
         'नीरज': 0.13353139262452257,
         'मर': 2.0794415416798357,
         'दास': 2.0794415416798357,
         'आत्मविश्वास': 2.0794415416798357,
         'सकूंगा': 2.0794415416798357,
         'हिंदी': 1.3862943611198906,
         'अंदर': 1.3862943611198906,
         'पुस्तक': 2.0794415416798357,
         'बहाने': 2.0794415416798357,
         'स्वयं': 2.0794415416798357,
         'सपनो': 2.0794415416798357,
         'पढ़कर': 2.0794415416798357,
         'परिचय': 2.0794415416

In [189]:
con1 = WXC(order='wx2utf') 
con1.convert('mara')

'मर'

In [251]:
vocab=set(reduce(lambda x,y:x+y,tokenized_list))
wordDict = dict.fromkeys(vocab, 0) 


tfword=[computeTf(wordDict.copy(),sent) for sent in tokenized_list]
docList=[i[1] for i in tfword]
tf=[i[0] for i in tfword]
idfs=computeIDF(docList)


tfidf=[computeTFIDF(tf_i, idfs) for tf_i in tf]
tfidf_average = [sum(j.values())/len([e for e in j.values() if e]) for j in tfidf]

In [252]:
tfidf_average


[0.012133943116785863,
 0.017391387694478136,
 0.04626387040077326,
 0.020283215710739017,
 0.03973620601272673,
 0.04868028327937198,
 0.0513253720776522,
 0.0190496180175016]

In [295]:
import re
data="dsmk kks 7"
pattern=re.compile(r'((?:(?:(?:(?:(?:[012]\d{1})|[1-9])[-\/]){2}(?:[12]\d{3}|\d{2}))|(?:[01]?\d{1}[-\/])(?:30|31)[-\/](?:[12]\d{3}|\d{2})|(?:(?:[012]\d{1})|[1-9])[-\/][12]\d{3}|(?:(?:\d{1,2}\s)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:\.|,|\s|-)?(?:\d{1,2})?(?:st|th|rd|nd)?(?:,|\s|-)?\s?[[12]\d{3}|\d{2}])|(?:(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:\.|,|\s|-)?\s?(?:\d{1,2})(?:st|th|rd|nd)?(?:,|\s|-)?\s?[[12]\d{3}|\d{2}]))|[12]{1}\d{3})')
re.findall(pattern, data)

[]

In [310]:
def sent_position():
    l=len(tokenized_list)
    position=[0]*l
    if l%2==0:
        mid=int(l/2)
    else:
        mid=int((l+1)/2 )  
    for i in range(0,mid):
        position[i]=l-i
        position[l-i-1]=l-i
    return(position)

In [311]:
sent_position()

[9, 8, 7, 6, 5, 6, 7, 8, 9]

In [1]:
with open("support_data/english_romancized.txt","r") as g:
    d=set(g.read().strip().splitlines())

In [2]:
d.discard("\n")

In [3]:
df=[]
for i in d:
    df.append(i.split(",")[0]+"a")

In [4]:
len(df)

570

In [ ]:
for i in df 

In [8]:
con = WXC(order='utf2wx') 
con1 = WXC(order='wx2utf')
data_engl=[]
for i in df:
    t=i.split()
    if len(t)>1:
        for k in t[:-1]:
            l=con1.convert(k+"a")
            data_engl.append(l)
        l=con1.convert(t[-1])
        data_engl.append(l)
    else:
        l=con1.convert(i)
        data_engl.append(l)

In [9]:
data_engl=set(data_engl)

In [350]:
data_engl

{'(ए.ग्.अ',
 '(एन्डेएटेएवेए)अ',
 '(ऐ/H)अ',
 '(क्हेलोण्)अ',
 '(नोठ्स्)अ',
 'अ',
 'अअ',
 'अअकअस्ह',
 'अअप्रेस्हन',
 'अअम्लेट',
 'अअर्किटेक्ट',
 'अअर्केस्ट्रअअ',
 'अअर्जेन्टेएनअअ',
 'अइप',
 'अइस्क्रेएम',
 'अइस्च्रिम',
 'अइस्ड',
 'अउङ',
 'अउट',
 'अउट्!अ',
 'अउप्रेस्हन',
 'अउप्ह',
 'अउर्ढर',
 'अएए.ए.एस्स्.अ',
 'अएए.एस्.अएए.अ',
 'अएए.सेए.योओ.अ',
 'अएए.सेए.सेए.अ',
 'अएएअएएटेएअ',
 'अकउण्ट',
 'अक्टोओबर',
 'अक्सिजअन',
 'अटक',
 'अट्होरिटेएअ',
 'अपेएल',
 'अप्रइल',
 'अम्पयर',
 'अल्टेएमेटम',
 'इण्कम',
 'इण्ग्लइण्ड',
 'इण्ग्लइन्ड',
 'इण्ङउर्म',
 'इण्जनेएयर',
 'इण्जेएनियर',
 'इण्जेक्स्हन',
 'इण्टर्नट',
 'इण्टर्नेट',
 'इण्डइक्स',
 'इण्डस्ट्रेएअ',
 'इण्डियअअ',
 'इण्ढियअअ',
 'इण्वइट',
 'इन्ङोर्म',
 'इन्प्ह्लेस्हन',
 'इमेज',
 'इयर्पोर्ट',
 'इलक्ट्रनिक',
 'ईन्डेएयअअ',
 'उन्ङेर',
 'एए-मेल',
 'एएरक',
 'एकउण्ठ',
 'एक्टर्स',
 'एक्पट्रअअ',
 'एक्स्प्रेस',
 'एक्स्रेअ',
 'एङ्डेएअएएअ',
 'एच्हइवेएअ',
 'एजेन्सेएअ',
 'एट्च्.अ',
 'एठेएएम',
 'एडिटर',
 'एडिस्हअन',
 'एड्मिस्हन',
 'एड्स',
 'एन्.डेए.ए.अ',
 'एन्.डेए.टेए.वेए.

In [10]:
data_engl.discard("अ")
data_engl.discard("अअ")

In [11]:
with open("support_data/English_loanwords.txt","r") as g:
    k=g.read().strip().splitlines()

In [13]:
with open("support_data/eng_loan.txt","r") as g:
    k=g.read().strip().splitlines()

In [12]:
english_loanwords = set(k).union(data_engl)

In [14]:
english_loanwords = set(k).union(english_loanwords)

In [15]:
len(english_loanwords)

1099

In [376]:
inputString = ["dk","7","3","fd","fg"]
[1 if char.isdigit() else 0 for char in inputString]

[0, 1, 1, 0, 0]